## Importing libraries and reading dataset

In [340]:
import pandas as pd
import numpy as np
import fastparquet
import pyarrow

In [341]:
df=pd.read_csv('../data/raw/BreastCancerDS.csv',index_col=0)


In [342]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19710 entries, 0 to 19709
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    19710 non-null  int64 
 1   perimeter_se             19015 non-null  object
 2   radius_worst             19085 non-null  object
 3   concave points_mean      18980 non-null  object
 4   smoothness_mean          18825 non-null  object
 5   area_mean                18830 non-null  object
 6   concavity_se             18905 non-null  object
 7   texture_mean             18955 non-null  object
 8   concavity_worst          19055 non-null  object
 9   smoothness_se            19110 non-null  object
 10  concave points_se        19000 non-null  object
 11  area_worst               19060 non-null  object
 12  compactness_mean         18935 non-null  object
 13  radius_mean              19000 non-null  object
 14  area_se                  18895 non-nul

## Data description

completar

## Cleaning target feature, we'll only use examples wich target is correctly labeled, in this case B ( benign) or M (Malignant)
* Before target cleaning we had 19710 entries, once the target is cleaned, we have 16180 entries.

In [343]:
df=df[(df['diagnosis']=='B')|(df['diagnosis']=='M')]


## Deleting iuytr y erty columns
* 'Erty' Column contains the same value for each row, we can delete it since it provides no information.
* 'iuytr' column contains the same information in 'symmetry_mean' we delete one of them, in this case 'iuytr' is deleted.

In [344]:
df['erty'].value_counts() ## todos los valores son iguales, se borra erty


908765434567    16180
Name: erty, dtype: int64

In [345]:
df.drop('erty',axis=1,inplace=True)

In [346]:
df[['iuytr','symmetry_mean']] ## estas dos columnas tienen la misma informacion, borrar una


,iuytr,symmetry_mean
1,211.0,211.0
2,-88888765432345.0,-88888765432345.0
4,?,?
5,0.1671,0.1671
7,0.1667,0.1667
...,...,...
19703,216.0,216.0
19705,0.1671,0.1671
19706,0.1405,0.1405
19708,0.2275,0.2275


In [347]:
df.drop('iuytr',axis=1,inplace=True)

In [348]:
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,999765432456788.0,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,8911834,211.0,0.2955,B,0.001593,999765432456788.0,0.1724,0.01528,0.01541,999765432456788.0
2,307,1144.0,9699.0,0.003472,-88888765432345.0,246.3,0.003681,14.4,0.01472,0.007389,...,89346,-88888765432345.0,0.2991,B,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,?,0.02278,0.09524,409.0,0.01349,18.75,?,0.008328,...,85759902,?,0.3306,B,0.002386,73.34,?,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,899187,0.1671,0.2731,B,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,?,...,865137,0.1667,0.3016,B,0.002206,73.34,239.0,0.01344,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,?,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,86730502,216.0,?,M,?,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,rxctf378968 7656463sdfg,0.01162,0.07561,421.0,0.005949,17.07,0.03046,rxctf378968 7656463sdfg,...,899187,0.1671,0.2731,B,rxctf378968 7656463sdfg,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,-88888765432345.0,585.9,0.007741,15.79,0.03517,-88888765432345.0,...,90944601,0.1405,0.1859,B,0.002564,-88888765432345.0,0.1071,-88888765432345.0,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,91504,0.2275,0.3651,M,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


## Identifying missing values
We found around 3-4% of na values for each feature, we'll analize what to do with these features after a more in depth analisis. See below.

In [349]:
missing = df.isnull().sum()
missing[missing>0]*100/len(df)

perimeter_se               3.275649
radius_worst               3.182942
concave points_mean        3.244747
smoothness_mean            4.295426
area_mean                  4.140915
concavity_se               3.831891
texture_mean               3.522868
concavity_worst            3.213844
smoothness_se              2.626700
concave points_se          3.430161
area_worst                 3.337454
compactness_mean           3.770087
radius_mean                3.491965
area_se                    3.831891
concave points_worst       2.997528
fractal_dimension_worst    2.904821
perimeter_worst            3.152040
texture_se                 3.182942
fractal_dimension_mean     3.090235
texture_worst              3.244747
smoothness_worst           3.337454
concavity_mean             4.295426
id                         2.657602
symmetry_mean              3.831891
symmetry_worst             2.843016
fractal_dimension_se       3.059333
perimeter_mean             3.770087
compactness_worst          2

## Cleaning spaces after and before data
* At this point we need to check if our data has spaces at the beggining or at the end, in order to do this, we change index type to string and use a for cicle to apply str.strip() function to all the columns, so the spaces after and before each string are deleted. We'll change datatypes to numeric again as they are supossed to be,but this allows us to handle possible mistakes when cleaning.

In [350]:
df['index']=df['index'].astype('string')
list= ['index', 'perimeter_se', 'radius_worst', 'concave points_mean',
       'smoothness_mean', 'area_mean', 'concavity_se', 'texture_mean',
       'concavity_worst', 'smoothness_se', 'concave points_se', 'area_worst',
       'compactness_mean', 'radius_mean', 'area_se', 'concave points_worst'
       , 'fractal_dimension_worst', 'perimeter_worst', 'texture_se',
       'fractal_dimension_mean', 'texture_worst', 'smoothness_worst',
       'concavity_mean', 'id', 'symmetry_mean', 'symmetry_worst', 'diagnosis',
       'fractal_dimension_se', 'perimeter_mean', 'compactness_worst',
       'symmetry_se', 'compactness_se', 'radius_se']
for i in list:
    df[i]= df[i].str.strip()

## Handling corrupt values
* Checking the whole dataset, we can see there are 4 types of errors present in many (almost all) columns, we create a list with these errors and use a for to replace these errors for np.nan in each column of our dataset (df) in order to better handle possible mistakes using methods for np.nan.

 Errors found:
- 'rxctf378968 7656463sdfg'
- '-88888765432345.0'
- '999765432456788.0'
- '?'

In [351]:
list=['rxctf378968 7656463sdfg','-88888765432345.0','999765432456788.0','?']
for i in list:
       df.replace(i,np.nan,inplace=True)


In [352]:
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,NaN,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,8911834,211.0,0.2955,B,0.001593,NaN,0.1724,0.01528,0.01541,NaN
2,307,1144.0,9699.0,0.003472,NaN,246.3,0.003681,14.4,0.01472,0.007389,...,89346,NaN,0.2991,B,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,NaN,0.02278,0.09524,409.0,0.01349,18.75,NaN,0.008328,...,85759902,NaN,0.3306,B,0.002386,73.34,NaN,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,899187,0.1671,0.2731,B,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,NaN,...,865137,0.1667,0.3016,B,0.002206,73.34,239.0,0.01344,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,NaN,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,86730502,216.0,NaN,M,NaN,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,NaN,0.01162,0.07561,421.0,0.005949,17.07,0.03046,NaN,...,899187,0.1671,0.2731,B,NaN,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,NaN,585.9,0.007741,15.79,0.03517,NaN,...,90944601,0.1405,0.1859,B,0.002564,NaN,0.1071,NaN,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,91504,0.2275,0.3651,M,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


## Enconding target
We use scikit learn LabelEncoder to encode diagnosis as follows:
0 if diagnosis is B (Beningn) and  1 if its M (malignant)

In [353]:
from sklearn import preprocessing
label_encoding = preprocessing.LabelEncoder()
df['diagnosis'] = label_encoding.fit_transform(df['diagnosis'])

## 0 es beningno y 1 es maligno


## Seting dataset dtype as float and diagnosis as category

In [354]:
df=df.astype('float')

In [355]:
df['diagnosis']=df['diagnosis'].astype('int').astype('category')


In [356]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16180 entries, 1 to 19709
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   index                    16180 non-null  float64 
 1   perimeter_se             13640 non-null  float64 
 2   radius_worst             13630 non-null  float64 
 3   concave points_mean      13600 non-null  float64 
 4   smoothness_mean          13450 non-null  float64 
 5   area_mean                13425 non-null  float64 
 6   concavity_se             13475 non-null  float64 
 7   texture_mean             13565 non-null  float64 
 8   concavity_worst          13540 non-null  float64 
 9   smoothness_se            13610 non-null  float64 
 10  concave points_se        13600 non-null  float64 
 11  area_worst               13530 non-null  float64 
 12  compactness_mean         13480 non-null  float64 
 13  radius_mean              13495 non-null  float64 
 14  area_s

## First describe
We can now make a first description of the raw data

In [357]:
df.describe()

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,concavity_mean,id,symmetry_mean,symmetry_worst,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
count,16180.00000,13640.000000,13630.000000,13600.000000,13450.000000,13425.000000,13475.000000,13565.000000,13540.000000,13610.000000,...,13415.000000,1.472000e+04,13485.000000,13600.000000,13555.000000,13450.000000,13725.000000,13580.000000,13570.000000,13595.000000
mean,328.05068,2536.409364,323.328936,2.424616,3.835104,656.435978,1.147903,19.416823,24.441271,0.007034,...,7.370354,3.075898e+13,16.305977,31.056838,0.010388,92.300212,25.303201,0.219976,0.145002,81.108863
std,189.47624,1738.237381,1680.786074,16.207594,19.928257,349.181590,17.733254,4.385227,106.068910,0.003130,...,34.793982,1.824612e+14,52.516520,92.196651,0.199383,24.473417,95.437704,2.066789,1.474177,292.899986
min,0.00000,0.771400,7.930000,0.000000,0.052630,143.500000,0.000000,9.710000,0.000000,0.001713,...,0.000000,-8.888877e+13,0.116700,0.156500,0.000895,43.790000,0.027290,0.007882,0.002252,0.111500
25%,165.75000,1482.000000,13.290000,0.020270,0.086410,420.500000,0.015140,16.330000,0.121100,0.005033,...,0.029950,8.654680e+05,0.163400,0.252300,0.002217,75.490000,0.150800,0.015020,0.013400,0.235100
50%,326.00000,2143.000000,15.300000,0.033700,0.095920,556.700000,0.026260,18.900000,0.257100,0.006307,...,0.063870,9.079150e+05,0.181300,0.288400,0.003053,86.910000,0.229700,0.018525,0.020480,0.343800
75%,494.00000,3168.000000,19.920000,0.077520,0.106100,782.700000,0.042560,21.870000,0.426725,0.008109,...,0.145700,8.912944e+06,0.202700,0.331300,0.004463,104.300000,0.385600,0.023240,0.032470,0.590700
max,656.00000,9807.000000,9981.000000,162.000000,123.000000,2501.000000,396.000000,39.280000,1252.000000,0.031130,...,313.000000,9.997654e+14,304.000000,544.000000,6.000000,188.500000,1058.000000,31.000000,27.000000,2873.000000


In [358]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16180 entries, 1 to 19709
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   index                    16180 non-null  float64 
 1   perimeter_se             13640 non-null  float64 
 2   radius_worst             13630 non-null  float64 
 3   concave points_mean      13600 non-null  float64 
 4   smoothness_mean          13450 non-null  float64 
 5   area_mean                13425 non-null  float64 
 6   concavity_se             13475 non-null  float64 
 7   texture_mean             13565 non-null  float64 
 8   concavity_worst          13540 non-null  float64 
 9   smoothness_se            13610 non-null  float64 
 10  concave points_se        13600 non-null  float64 
 11  area_worst               13530 non-null  float64 
 12  compactness_mean         13480 non-null  float64 
 13  radius_mean              13495 non-null  float64 
 14  area_s

## Saving current data into a parquet file

In [359]:
df.to_parquet("../data/interim/BreastCancer.parquet", index = False)


In [360]:

df=pd.read_parquet("../data/interim/BreastCancer.parquet", engine='pyarrow')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16180 entries, 0 to 16179
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    16180 non-null  float64
 1   perimeter_se             13640 non-null  float64
 2   radius_worst             13630 non-null  float64
 3   concave points_mean      13600 non-null  float64
 4   smoothness_mean          13450 non-null  float64
 5   area_mean                13425 non-null  float64
 6   concavity_se             13475 non-null  float64
 7   texture_mean             13565 non-null  float64
 8   concavity_worst          13540 non-null  float64
 9   smoothness_se            13610 non-null  float64
 10  concave points_se        13600 non-null  float64
 11  area_worst               13530 non-null  float64
 12  compactness_mean         13480 non-null  float64
 13  radius_mean              13495 non-null  float64
 14  area_se               

In [361]:
## BORRAR SOLO PROVISIONAL
df.dropna(inplace=True)

In [362]:
df['diagnosis']=df['diagnosis'].astype('category')

In [363]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3230 entries, 3 to 16178
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   index                    3230 non-null   float64 
 1   perimeter_se             3230 non-null   float64 
 2   radius_worst             3230 non-null   float64 
 3   concave points_mean      3230 non-null   float64 
 4   smoothness_mean          3230 non-null   float64 
 5   area_mean                3230 non-null   float64 
 6   concavity_se             3230 non-null   float64 
 7   texture_mean             3230 non-null   float64 
 8   concavity_worst          3230 non-null   float64 
 9   smoothness_se            3230 non-null   float64 
 10  concave points_se        3230 non-null   float64 
 11  area_worst               3230 non-null   float64 
 12  compactness_mean         3230 non-null   float64 
 13  radius_mean              3230 non-null   float64 
 14  area_se

In [364]:
df=df.drop_duplicates(keep='first', ignore_index=False)
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
3,350.0,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,899187.0,0.1671,0.2731,0,0.002668,73.70,0.06476,0.02216,0.006991,0.3534
13,562.0,2362.0,17.52,0.09429,0.10480,716.9,0.073590,30.62,1.17000,0.004625,...,925622.0,0.2128,0.4089,1,0.006142,103.40,0.79170,0.02137,0.048440,0.2602
18,312.0,2346.0,14.19,0.02548,0.08794,504.1,0.031370,13.37,0.17690,0.006494,...,89382602.0,0.1601,0.2564,0,0.004392,82.29,0.22080,0.01731,0.027680,0.3265
22,341.0,1429.0,10.75,0.02292,0.08481,280.5,0.050280,16.84,0.43410,0.005954,...,898143.0,0.2036,0.2982,0,0.004031,61.64,0.34160,0.01750,0.034710,0.1844
45,8.0,2406.0,15.49,0.09353,0.12730,519.8,0.035530,21.82,539.00000,0.005731,...,844981.0,235.0000,0.4378,1,0.003749,87.50,0.54010,0.02143,0.035020,0.3063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,408.0,3061.0,21.08,0.08824,0.10360,991.7,0.025090,20.66,0.33010,0.007231,...,90524101.0,0.1992,306.0000,1,0.003336,117.80,0.37350,0.01414,0.027720,0.4537
3214,635.0,1657.0,14.34,0.01762,0.08369,520.2,0.005681,25.13,0.04462,0.006054,...,9112594.0,0.1667,0.2306,0,0.001514,82.61,0.10930,0.01215,0.008974,0.2621
3220,184.0,1344.0,17.80,0.03263,0.09057,710.6,0.017740,22.41,363.00000,0.003290,...,873885.0,0.1727,0.3175,1,0.002575,98.92,0.32990,0.01172,0.013950,0.2054
3221,561.0,2041.0,11.92,0.00000,0.07449,386.0,0.000000,29.37,0.00000,0.007594,...,925311.0,106.0000,0.1566,0,0.001773,70.67,0.05494,0.01989,0.008878,0.3141


In [365]:
df=df.drop('id',axis=1)
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,concavity_mean,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
3,350.0,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,0.008306,0.1671,0.2731,0,0.002668,73.70,0.06476,0.02216,0.006991,0.3534
13,562.0,2362.0,17.52,0.09429,0.10480,716.9,0.073590,30.62,1.17000,0.004625,...,255.000000,0.2128,0.4089,1,0.006142,103.40,0.79170,0.02137,0.048440,0.2602
18,312.0,2346.0,14.19,0.02548,0.08794,504.1,0.031370,13.37,0.17690,0.006494,...,0.040520,0.1601,0.2564,0,0.004392,82.29,0.22080,0.01731,0.027680,0.3265
22,341.0,1429.0,10.75,0.02292,0.08481,280.5,0.050280,16.84,0.43410,0.005954,...,0.084220,0.2036,0.2982,0,0.004031,61.64,0.34160,0.01750,0.034710,0.1844
45,8.0,2406.0,15.49,0.09353,0.12730,519.8,0.035530,21.82,539.00000,0.005731,...,0.185900,235.0000,0.4378,1,0.003749,87.50,0.54010,0.02143,0.035020,0.3063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,408.0,3061.0,21.08,0.08824,0.10360,991.7,0.025090,20.66,0.33010,0.007231,...,0.120100,0.1992,306.0000,1,0.003336,117.80,0.37350,0.01414,0.027720,0.4537
3214,635.0,1657.0,14.34,0.01762,0.08369,520.2,0.005681,25.13,0.04462,0.006054,...,0.012060,0.1667,0.2306,0,0.001514,82.61,0.10930,0.01215,0.008974,0.2621
3220,184.0,1344.0,17.80,0.03263,0.09057,710.6,0.017740,22.41,363.00000,0.003290,...,0.053750,0.1727,0.3175,1,0.002575,98.92,0.32990,0.01172,0.013950,0.2054
3221,561.0,2041.0,11.92,0.00000,0.07449,386.0,0.000000,29.37,0.00000,0.007594,...,0.000000,106.0000,0.1566,0,0.001773,70.67,0.05494,0.01989,0.008878,0.3141


In [366]:
df=df.drop_duplicates(keep='first', ignore_index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 641 entries, 3 to 3234
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   index                    641 non-null    float64 
 1   perimeter_se             641 non-null    float64 
 2   radius_worst             641 non-null    float64 
 3   concave points_mean      641 non-null    float64 
 4   smoothness_mean          641 non-null    float64 
 5   area_mean                641 non-null    float64 
 6   concavity_se             641 non-null    float64 
 7   texture_mean             641 non-null    float64 
 8   concavity_worst          641 non-null    float64 
 9   smoothness_se            641 non-null    float64 
 10  concave points_se        641 non-null    float64 
 11  area_worst               641 non-null    float64 
 12  compactness_mean         641 non-null    float64 
 13  radius_mean              641 non-null    float64 
 14  area_se  

In [367]:
df=df.drop('index',axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 641 entries, 3 to 3234
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   perimeter_se             641 non-null    float64 
 1   radius_worst             641 non-null    float64 
 2   concave points_mean      641 non-null    float64 
 3   smoothness_mean          641 non-null    float64 
 4   area_mean                641 non-null    float64 
 5   concavity_se             641 non-null    float64 
 6   texture_mean             641 non-null    float64 
 7   concavity_worst          641 non-null    float64 
 8   smoothness_se            641 non-null    float64 
 9   concave points_se        641 non-null    float64 
 10  area_worst               641 non-null    float64 
 11  compactness_mean         641 non-null    float64 
 12  radius_mean              641 non-null    float64 
 13  area_se                  641 non-null    float64 
 14  concave p

In [368]:
df=df.drop_duplicates(keep='first', ignore_index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569 entries, 3 to 3234
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   perimeter_se             569 non-null    float64 
 1   radius_worst             569 non-null    float64 
 2   concave points_mean      569 non-null    float64 
 3   smoothness_mean          569 non-null    float64 
 4   area_mean                569 non-null    float64 
 5   concavity_se             569 non-null    float64 
 6   texture_mean             569 non-null    float64 
 7   concavity_worst          569 non-null    float64 
 8   smoothness_se            569 non-null    float64 
 9   concave points_se        569 non-null    float64 
 10  area_worst               569 non-null    float64 
 11  compactness_mean         569 non-null    float64 
 12  radius_mean              569 non-null    float64 
 13  area_se                  569 non-null    float64 
 14  concave p

In [369]:
df.to_parquet("../data/interim/Breastclean1.parquet", index = False)
